In [1]:
import subprocess, textwrap

print("Où Windows trouve hadoop.dll ?")
res = subprocess.run(["where", "hadoop.dll"], capture_output=True, text=True)
print(res.stdout if res.stdout else res.stderr)

Où Windows trouve hadoop.dll ?
Information : impossible de trouver des fichiers pour le(s) modèle(s) spécifié(s).



# Définir HADOOP_HOME/PATH

In [2]:
import os
from pathlib import Path

HADOOP_HOME = (Path.cwd() / "hadoop").resolve()
BIN = HADOOP_HOME / "bin"

WINUTILS = BIN / "winutils.exe"
HADOOP_DLL = BIN / "hadoop.dll"

assert WINUTILS.exists(), f"winutils.exe introuvable: {WINUTILS}"
assert HADOOP_DLL.exists(), f"hadoop.dll introuvable: {HADOOP_DLL}"

os.environ["HADOOP_HOME"] = str(HADOOP_HOME)
os.environ["hadoop.home.dir"] = str(HADOOP_HOME)
os.environ["PATH"] = str(BIN) + os.pathsep + os.environ.get("PATH", "")

print("HADOOP_HOME =", os.environ["HADOOP_HOME"])
print("BIN =", BIN)

HADOOP_HOME = C:\Users\ilyes\M2\Algorithmique\Projet\hadoop
BIN = C:\Users\ilyes\M2\Algorithmique\Projet\hadoop\bin


# Paramètres + vérification des fichiers

In [3]:
# Cellule 1 — Paramètres + vérification des fichiers
import os

RAW_ADS_PATH = "idf_ventes.csv"
CITIES_PATH  = "cities.csv"

# Dataset final sauvegardé (format Parquet recommandé)
OUTPUT_DATASET_PATH = "dataset_final_immo_idf_parquet"

print("CWD:", os.getcwd())
print("Fichiers présents:", os.listdir("."))

assert os.path.exists(RAW_ADS_PATH), f"Fichier introuvable: {RAW_ADS_PATH}"
assert os.path.exists(CITIES_PATH),  f"Fichier introuvable: {CITIES_PATH}"

CWD: C:\Users\ilyes\M2\Algorithmique\Projet
Fichiers présents: ['.ipynb_checkpoints', 'apprentissage_pyspark.ipynb', 'cities.csv', 'dataset_final_immo_idf_parquet', 'hadoop', 'idf_ventes.csv', 'nettoyage_pyspark.ipynb', 'scrap.py']


# Démarrage Spark

In [4]:
# Cellule 2 — Démarrage Spark
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("ImmoIDF-Nettoyage")
         .getOrCreate())

# Lecture du CSV des annonces + trim + contrôles

In [5]:
# Cellule 3 — Lecture du CSV annonces + trim + contrôles
from pyspark.sql import functions as F

EXPECTED_COLS = ["Ville", "Type", "Surface", "NbrPieces", "NbrChambres", "NbrSdb", "DPE", "Prix"]

annonces_raw = (
    spark.read
    .option("header", True)
    .option("encoding", "utf-8")
    .csv(RAW_ADS_PATH)
)

# Trim sur toutes les colonnes
annonces = annonces_raw
for c in annonces.columns:
    annonces = annonces.withColumn(c, F.trim(F.col(c)))

print("Colonnes:", annonces.columns)
missing = [c for c in EXPECTED_COLS if c not in annonces.columns]
assert len(missing) == 0, f"Colonnes manquantes dans {RAW_ADS_PATH}: {missing}"

print("Schéma (brut):")
annonces.printSchema()

print("Aperçu:")
annonces.show(10, truncate=False)

print("Nb lignes:", annonces.count())

Colonnes: ['Ville', 'Type', 'Surface', 'NbrPieces', 'NbrChambres', 'NbrSdb', 'DPE', 'Prix']
Schéma (brut):
root
 |-- Ville: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Surface: string (nullable = true)
 |-- NbrPieces: string (nullable = true)
 |-- NbrChambres: string (nullable = true)
 |-- NbrSdb: string (nullable = true)
 |-- DPE: string (nullable = true)
 |-- Prix: string (nullable = true)

Aperçu:
+------------------+------+-------+---------+-----------+------+------+------+
|Ville             |Type  |Surface|NbrPieces|NbrChambres|NbrSdb|DPE   |Prix  |
+------------------+------+-------+---------+-----------+------+------+------+
|Paris 15ème       |Maison|220    |8        |4          |3     |Vierge|250000|
|Meaux             |Maison|325    |10       |5          |-     |-     |749000|
|Misy-sur-Yonne    |Maison|128    |5        |4          |1     |E     |279000|
|Nanteuil-lès-Meaux|Maison|89     |5        |3          |-     |F     |242000|
|Villeparisis      |M

# Nettoyage des catégories (Type, DPE)

In [6]:
# Cellule 4 — Nettoyage catégories (Type, DPE)
# Objectif :
# - Type ∈ {"Maison","Appartement"} sinon null
# - DPE ∈ {"A","B","C","D","E","F","G"} sinon "Vierge"
annonces = (annonces
    .withColumn("Type", F.when(F.col("Type").isin("Maison", "Appartement"), F.col("Type")).otherwise(F.lit(None)))
    .withColumn("DPE", F.upper(F.col("DPE")))
    .withColumn("DPE", F.when(F.col("DPE").isin("A","B","C","D","E","F","G"), F.col("DPE")).otherwise(F.lit("Vierge")))
)

# On enlève les lignes sans Type valide (sinon vos dummies Type seront incohérentes)
annonces = annonces.dropna(subset=["Type"])

print("Types distincts:")
annonces.select("Type").distinct().show(truncate=False)

print("DPE distincts:")
annonces.select("DPE").distinct().show(truncate=False)

Types distincts:
+-----------+
|Type       |
+-----------+
|Appartement|
|Maison     |
+-----------+

DPE distincts:
+------+
|DPE   |
+------+
|F     |
|E     |
|B     |
|D     |
|C     |
|Vierge|
|G     |
+------+



# Nettoyage des numériques (Surface, pièces, chambres, sdb, prix) + cast

In [7]:
# Cellule 5 — Nettoyage numériques + cast
from pyspark.sql.types import DoubleType

num_cols = ["Surface", "NbrPieces", "NbrChambres", "NbrSdb", "Prix"]

for c in num_cols:
    annonces = (annonces
        .withColumn(
            c,
            F.when((F.col(c) == "-") | (F.col(c) == "") | F.col(c).isNull(), F.lit(None))
             .otherwise(F.col(c))
        )
        .withColumn(c, F.col(c).cast(DoubleType()))
    )

print("Schéma après cast:")
annonces.printSchema()

print("Null counts (numériques):")
annonces.select([F.count(F.when(F.col(c).isNull(), 1)).alias(f"null_{c}") for c in num_cols]).show(truncate=False)

Schéma après cast:
root
 |-- Ville: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Surface: double (nullable = true)
 |-- NbrPieces: double (nullable = true)
 |-- NbrChambres: double (nullable = true)
 |-- NbrSdb: double (nullable = true)
 |-- DPE: string (nullable = true)
 |-- Prix: double (nullable = true)

Null counts (numériques):
+------------+--------------+----------------+-----------+---------+
|null_Surface|null_NbrPieces|null_NbrChambres|null_NbrSdb|null_Prix|
+------------+--------------+----------------+-----------+---------+
|22          |21            |47              |86         |0        |
+------------+--------------+----------------+-----------+---------+



# Imputation par moyenne (numériques) + dropna résiduel

In [8]:
# Cellule 6 — Imputation par moyenne + dropna résiduel
means_row = annonces.agg(*[F.avg(F.col(c)).alias(c) for c in num_cols]).collect()[0]
means = {c: means_row[c] for c in num_cols}
print("Moyennes calculées:", means)

for c in num_cols:
    # Si la moyenne est None (colonne vide), on ne peut pas imputer -> on laissera en null
    if means[c] is not None:
        annonces = annonces.withColumn(c, F.when(F.col(c).isNull(), F.lit(means[c])).otherwise(F.col(c)))

# Si des nulls restent (ex: moyenne None), on supprime les lignes concernées
annonces = annonces.dropna(subset=num_cols)

print("Nb lignes après imputation/dropna:", annonces.count())

Moyennes calculées: {'Surface': 91.27362204724409, 'NbrPieces': 5.5265225933202355, 'NbrChambres': 2.525879917184265, 'NbrSdb': 1.2905405405405406, 'Prix': 1.4153816070518901e+29}
Nb lignes après imputation/dropna: 530


# Création des dummies Type et DPE (colonnes fixes)

In [9]:
# Cellule 7 — Création des dummies (colonnes fixes, stables)
# Type
annonces = (annonces
    .withColumn("Type_Maison",       F.when(F.col("Type") == "Maison", 1).otherwise(0).cast("int"))
    .withColumn("Type_Appartement", F.when(F.col("Type") == "Appartement", 1).otherwise(0).cast("int"))
)

# DPE : on garde B..G + Vierge (comme dans votre sélection finale)
for v in ["B","C","D","E","F","G","Vierge"]:
    annonces = annonces.withColumn(f"DPE_{v}", F.when(F.col("DPE") == v, 1).otherwise(0).cast("int"))

# Contrôle rapide
cols_to_show = ["Type","DPE","Type_Maison","Type_Appartement"] + [f"DPE_{v}" for v in ["B","C","D","E","F","G","Vierge"]]
annonces.select(cols_to_show).show(10, truncate=False)

+------+------+-----------+----------------+-----+-----+-----+-----+-----+-----+----------+
|Type  |DPE   |Type_Maison|Type_Appartement|DPE_B|DPE_C|DPE_D|DPE_E|DPE_F|DPE_G|DPE_Vierge|
+------+------+-----------+----------------+-----+-----+-----+-----+-----+-----+----------+
|Maison|Vierge|1          |0               |0    |0    |0    |0    |0    |0    |1         |
|Maison|Vierge|1          |0               |0    |0    |0    |0    |0    |0    |1         |
|Maison|E     |1          |0               |0    |0    |0    |1    |0    |0    |0         |
|Maison|F     |1          |0               |0    |0    |0    |0    |1    |0    |0         |
|Maison|B     |1          |0               |1    |0    |0    |0    |0    |0    |0         |
|Maison|B     |1          |0               |1    |0    |0    |0    |0    |0    |0         |
|Maison|E     |1          |0               |0    |0    |0    |1    |0    |0    |0         |
|Maison|C     |1          |0               |0    |1    |0    |0    |0    |0    |

# Lecture cities.csv (détection séparateur)

In [10]:
# Cellule 8 — Lecture cities.csv (détection séparateur)
villes = (spark.read
          .option("header", True)
          .option("encoding", "utf-8")
          .option("sep", ";")
          .csv(CITIES_PATH))

# Si une seule colonne, on retente avec ','
if len(villes.columns) == 1:
    villes = (spark.read
              .option("header", True)
              .option("encoding", "utf-8")
              .option("sep", ",")
              .csv(CITIES_PATH))

print("Colonnes villes:", villes.columns)
villes.show(5, truncate=False)

Colonnes villes: ['insee_code', 'city_code', 'zip_code', 'label', 'latitude', 'longitude', 'department_name', 'department_number', 'region_name', 'region_geojson_name']
+----------+-------------------+--------+-------------------+------------+-----------+---------------+-----------------+-----------------------+-----------------------+
|insee_code|city_code          |zip_code|label              |latitude    |longitude  |department_name|department_number|region_name            |region_geojson_name    |
+----------+-------------------+--------+-------------------+------------+-----------+---------------+-----------------+-----------------------+-----------------------+
|25620     |ville du pont      |25650   |ville du pont      |46.999873398|6.498147193|doubs          |25               |bourgogne-franche-comté|Bourgogne-Franche-Comté|
|25624     |villers grelot     |25640   |villers grelot     |47.361512085|6.235167025|doubs          |25               |bourgogne-franche-comté|Bourgogne-F

# UDF de normalisation des noms de ville (clé de jointure)

In [11]:
# Cellule 9 — Normalisation des villes (clé de jointure)
import unicodedata
from pyspark.sql.types import StringType

def strip_accents_py(s: str) -> str:
    if s is None:
        return None
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

strip_accents = F.udf(strip_accents_py, StringType())

def add_city_key(df, colname: str, out: str = "Ville_key"):
    x = F.lower(F.trim(F.col(colname)))
    x = strip_accents(x)
    x = F.regexp_replace(x, r"\b\d+\s*(er|e|eme|ème)\b", "")   # arrondissements
    x = F.regexp_replace(x, r"[^0-9a-z]+", " ")               # séparateurs -> espace
    x = F.regexp_replace(x, r"\s+", " ")
    x = F.trim(x)
    return df.withColumn(out, x)

# Filtre IDF + mapping Ville_key -> (lat, lon)

In [12]:
# Cellule 10 — Filtre IDF + mapping Ville_key -> (lat, lon)
IDF_DEPTS = ["75","77","78","91","92","93","94","95"]

villes_idf = villes.filter(F.col("department_number").isin(IDF_DEPTS))

villes_k = (add_city_key(villes_idf, "label", "Ville_key")
    .select(
        "Ville_key",
        F.col("latitude").cast("double").alias("latitude"),
        F.col("longitude").cast("double").alias("longitude"),
    )
    # Si plusieurs lignes pour une même ville, on moyenne
    .groupBy("Ville_key")
    .agg(F.avg("latitude").alias("latitude"), F.avg("longitude").alias("longitude"))
)

villes_k.show(10, truncate=False)

+--------------------+------------+-----------+
|Ville_key           |latitude    |longitude  |
+--------------------+------------+-----------+
|barbizon            |48.448347603|2.600809608|
|cergy               |49.039967587|2.051139021|
|coulommiers         |48.81234041 |3.091269785|
|reau                |48.60739655 |2.623860846|
|fontenay le vicomte |48.547225879|2.400248804|
|janvry              |48.646397715|2.15459332 |
|marolles en hurepoix|48.564568271|2.299591744|
|berville            |49.192304503|2.070134653|
|le plessis gassot   |49.035986122|2.414849753|
|lognes              |48.833863469|2.634038048|
+--------------------+------------+-----------+
only showing top 10 rows



# Jointure annonces ↔ villes + corrections de cas particuliers

In [13]:
# Cellule 11 — Jointure annonces ↔ villes + corrections de cas particuliers
annonces_tmp = add_city_key(annonces, "Ville", "Ville_key")

# Cas particuliers (comme dans votre notebook)
annonces_tmp = annonces_tmp.withColumn(
    "Ville_key",
    F.when(F.col("Ville_key").isin("evry", "courcouronnes"), F.lit("evry courcouronnes"))
     .when(F.col("Ville_key") == "le chesnay", F.lit("le chesnay rocquencourt"))
     .otherwise(F.col("Ville_key"))
)

annonces_geo = annonces_tmp.join(villes_k, on="Ville_key", how="left")

print("Nb lignes annonces_geo:", annonces_geo.count())
print("Nb lignes sans coordonnées:",
      annonces_geo.filter(F.col("latitude").isNull() | F.col("longitude").isNull()).count())

# On enlève celles sans coordonnées
annonces_geo = annonces_geo.dropna(subset=["latitude", "longitude"])
print("Nb lignes après suppression lat/lon manquants:", annonces_geo.count())

Nb lignes annonces_geo: 530
Nb lignes sans coordonnées: 27
Nb lignes après suppression lat/lon manquants: 503


# Sélection des colonnes finales (dataset ML) + cast + dropna

In [14]:
# Cellule 12 — Colonnes finales (dataset ML) + cast + dropna
from pyspark.sql.types import DoubleType, IntegerType

final_cols = [
    "Surface", "NbrPieces", "NbrChambres", "NbrSdb", "Prix",
    "DPE_B", "DPE_C", "DPE_D", "DPE_E", "DPE_F", "DPE_G", "DPE_Vierge",
    "Type_Appartement", "Type_Maison",
    "latitude", "longitude"
]

# Sécurité : si une colonne manque, on la crée à 0
for c in final_cols:
    if c not in annonces_geo.columns:
        annonces_geo = annonces_geo.withColumn(c, F.lit(0))

dataset_final = annonces_geo.select(*final_cols)

double_cols = ["Surface", "NbrPieces", "NbrChambres", "NbrSdb", "Prix", "latitude", "longitude"]
int_cols = [c for c in final_cols if c not in double_cols]

for c in double_cols:
    dataset_final = dataset_final.withColumn(c, F.col(c).cast(DoubleType()))
for c in int_cols:
    dataset_final = dataset_final.withColumn(c, F.col(c).cast(IntegerType()))

dataset_final = dataset_final.dropna()

print("Nb lignes (final):", dataset_final.count())
print("Nb colonnes (final):", len(dataset_final.columns))
dataset_final.printSchema()
dataset_final.show(10, truncate=False)

Nb lignes (final): 503
Nb colonnes (final): 16
root
 |-- Surface: double (nullable = true)
 |-- NbrPieces: double (nullable = true)
 |-- NbrChambres: double (nullable = true)
 |-- NbrSdb: double (nullable = true)
 |-- Prix: double (nullable = true)
 |-- DPE_B: integer (nullable = false)
 |-- DPE_C: integer (nullable = false)
 |-- DPE_D: integer (nullable = false)
 |-- DPE_E: integer (nullable = false)
 |-- DPE_F: integer (nullable = false)
 |-- DPE_G: integer (nullable = false)
 |-- DPE_Vierge: integer (nullable = false)
 |-- Type_Appartement: integer (nullable = false)
 |-- Type_Maison: integer (nullable = false)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

+-------+---------+-----------+------------------+---------+-----+-----+-----+-----+-----+-----+----------+----------------+-----------+------------+-----------+
|Surface|NbrPieces|NbrChambres|NbrSdb            |Prix     |DPE_B|DPE_C|DPE_D|DPE_E|DPE_F|DPE_G|DPE_Vierge|Type_Appartement|Type_Maiso

# Filtre “réaliste” sur le prix

In [15]:
# Cellule 13 — Filtre “réaliste” sur le prix
dataset_final_ml = dataset_final.filter(
    (F.col("Prix") >= 10_000) & (F.col("Prix") <= 5_000_000)
)

print("Lignes avant filtre :", dataset_final.count())
print("Lignes après filtre :", dataset_final_ml.count())

dataset_final_ml.select(
    F.min("Prix").alias("min_prix"),
    F.max("Prix").alias("max_prix")
).show(truncate=False)

Lignes avant filtre : 503
Lignes après filtre : 495
+--------+---------+
|min_prix|max_prix |
+--------+---------+
|14000.0 |4220000.0|
+--------+---------+



# Sauvegarde du dataset final (Parquet) + contrôle

In [16]:
# Cellule 14 — Sauvegarde du dataset final + contrôle
# Parquet (recommandé Spark)
(dataset_final_ml
 .write
 .mode("overwrite")
 .parquet(OUTPUT_DATASET_PATH))

print("Dataset sauvegardé en Parquet dans:", OUTPUT_DATASET_PATH)

# Contrôle : relecture
check = spark.read.parquet(OUTPUT_DATASET_PATH)
print("Relecture OK. Nb lignes:", check.count(), "Nb colonnes:", len(check.columns))
check.show(5, truncate=False)

Dataset sauvegardé en Parquet dans: dataset_final_immo_idf_parquet
Relecture OK. Nb lignes: 495 Nb colonnes: 16
+-------+---------+-----------+------+--------+-----+-----+-----+-----+-----+-----+----------+----------------+-----------+------------+-----------+
|Surface|NbrPieces|NbrChambres|NbrSdb|Prix    |DPE_B|DPE_C|DPE_D|DPE_E|DPE_F|DPE_G|DPE_Vierge|Type_Appartement|Type_Maison|latitude    |longitude  |
+-------+---------+-----------+------+--------+-----+-----+-----+-----+-----+-----+----------+----------------+-----------+------------+-----------+
|60.0   |3.0      |2.0        |1.0   |215000.0|0    |0    |0    |0    |0    |0    |1         |1               |0          |48.60739655 |2.623860846|
|70.0   |4.0      |2.0        |1.0   |180000.0|0    |0    |0    |0    |0    |0    |1         |1               |0          |48.657051798|2.38747617 |
|75.0   |2.0      |3.0        |1.0   |115000.0|0    |0    |0    |0    |0    |0    |1         |1               |0          |48.657051798|2.38747